In [95]:
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
import emnist
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import loader
mnist_images, mnist_labels = emnist.extract_test_samples('digits')
# animals_dataset = loader.load_chapel_dataset('./animals-10')

In [96]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=2),
    nn.Conv2d(in_channels=12,out_channels=16,kernel_size=4,stride=1),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.LazyLinear(out_features=10),
    nn.Softmax(dim=1)
)

In [97]:
model

Sequential(
  (0): Conv2d(1, 12, kernel_size=(3, 3), stride=(2, 2))
  (1): Conv2d(12, 16, kernel_size=(4, 4), stride=(1, 1))
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): LazyLinear(in_features=0, out_features=10, bias=True)
  (5): Softmax(dim=1)
)

In [98]:
img = mnist_images[0,:,:]
img = torch.from_numpy(img.reshape(1,1,28,28)).float()
img.shape

torch.Size([1, 1, 28, 28])

In [99]:
model(img).shape
model(img)

tensor([[2.2066e-12, 9.9853e-01, 3.2197e-38, 9.9211e-33, 2.8813e-23, 5.7358e-26,
         1.4662e-03, 9.5889e-17, 4.2346e-19, 7.2227e-24]],
       grad_fn=<SoftmaxBackward0>)

In [101]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=2),
    nn.Conv2d(in_channels=12,out_channels=16,kernel_size=4,stride=1),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.LazyLinear(out_features=10),
    nn.Softmax(dim=1)
)

epochs = 1
learning_rate = 0.005
batch_size = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0)

model.train()

for epoch in range(epochs):
    epoch_loss = 0
    num_pairs = mnist_images.shape[0]
    num_batches = num_pairs / batch_size
    with tqdm(total=num_pairs) as pbar:
        for (im_idx, label) in zip(range(num_pairs), mnist_labels):
            im = mnist_images[im_idx,:,:]
            im = torch.from_numpy(im.reshape(1,1,28,28)).float()

            optimizer.zero_grad()
            output = model(im)
            label_vector = torch.zeros(1,10)
            label_vector[0,label] = 1
            loss = loss_fn(output,label_vector)# -torch.sum(torch.log(output))
            loss.backward()
            optimizer.step()
            # print(loss.item(),output[0,label].item())
            epoch_loss += loss.item() 
            pbar.update(1)
    print('loss: ', epoch_loss / num_pairs)
    

/var/folders/wt/nr12w9r958d0nwkk86zwr9600000gp/T/ipykernel_6168/4022610662.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=num_pairs) as pbar:


  0%|          | 0/40000 [00:00<?, ?it/s]

loss:  2.3612710447967054
